In [1]:
#Reference:  https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/06_CIFAR-10.ipynb

%matplotlib inline
import numpy as np
import pickle
import os
import tarfile
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import prettytensor as pt

In [2]:
#data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
# Various constants for the size of the images.
# Width and height of each image.
img_size = 32
# Number of channels in each image
num_channels = 3
# Length of an image when flattened to a 1-dim array.
img_size_flat = img_size * img_size * num_channels
# Number of classes.
num_classes = 10
# Various constants used to allocate arrays of the correct size.
# Number of files for the training-set.
_num_files_train = 5
# Number of images for each batch-file in the training-set.
_images_per_file = 10000
# Total number of images in the training-set.
# This is used to pre-allocate arrays for efficiency.
_num_images_train = _num_files_train * _images_per_file

In [3]:
#Extracting data
# Assuming t
data_path='data/CIFAR-10'
tar_path='data/CIFAR-10/cifar-10-python.tar.gz'
with tarfile.open(tar_path) as tar:
    tar.extractall(path=data_path)
    tar.close()

In [4]:
#Return the full path of a data-file for the data-set
def _get_file_path(filename=""):
    return os.path.join(data_path, "cifar-10-batches-py/", filename)

In [5]:
#Load the names for the classes in the CIFAR-10 data-set. Returns a list with the names. 
def _unpickle(filename):
    file_path = _get_file_path(filename)
    with open(file_path, mode='rb') as file:
        data = pickle.load(file, encoding='bytes')

    return data

In [6]:
#Load the names for the classes in the CIFAR-10 data-set. Returns a list with the names. 
def load_class_names():
    # Load the class-names from the pickled file.
    raw = _unpickle(filename="batches.meta")[b'label_names']
    # Convert from binary strings.
    names = [x.decode('utf-8') for x in raw]
    return names

In [7]:
#Convert images from the CIFAR-10 format and return a 4-dim array with shape: [image_number, height, width, channel]
def _convert_images(raw):
    # Convert the raw images from the data-files to floating-points.
    raw_float = np.array(raw, dtype=float) / 255.0
    # Reshape the array to 4-dimensions.
    images = raw_float.reshape([-1, num_channels, img_size, img_size])
    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])
    return images

In [8]:
# Load a pickled data-file from the CIFAR-10 data-set and return the converted images (see above) and the class-number for each image.
def _load_data(filename):
    # Load the pickled data-file.
    data = _unpickle(filename)
    # Get the raw images.
    raw_images = data[b'data']
    # Get the class-numbers for each image. Convert to numpy-array.
    cls = np.array(data[b'labels'])
    # Convert the images.
    images = _convert_images(raw_images)
    return images, cls

In [9]:
# Load all the training-data for the CIFAR-10 data-set. The data-set is split into 5 data-files which are merged here.
# Returns the images, class-numbers and one-hot encoded class-labels.
def load_training_data():
    # Pre-allocate the arrays for the images and class-numbers for efficiency.
    images = np.zeros(shape=[_num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[_num_images_train], dtype=int)
    # Begin-index for the current batch.
    begin = 0
    # For each data-file.
    for i in range(_num_files_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = _load_data(filename="data_batch_" + str(i + 1))
        # Number of images in this batch.
        num_images = len(images_batch)
        # End-index for the current batch.
        end = begin + num_images
        # Store the images into the array.
        images[begin:end, :] = images_batch
        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch
        # The begin-index for the next batch is the current end-index.
        begin = end
    return images, cls, one_hot_encoded(class_numbers=cls, num_classes=num_classes)

In [10]:
# Load all the test-data for the CIFAR-10 data-set. Returns the images, class-numbers and one-hot encoded class-labels.
def load_test_data():
    images, cls = _load_data(filename="test_batch")
    return images, cls, one_hot_encoded(class_numbers=cls, num_classes=num_classes)

In [11]:
#Generate the One-Hot encoded class-labels from an array of integers.
def one_hot_encoded(class_numbers, num_classes=None):
    # Find the number of classes if None is provided.
    # Assumes the lowest class-number is zero.
    if num_classes is None:
        num_classes = np.max(class_numbers) + 1
    return np.eye(num_classes, dtype=float)[class_numbers]

In [12]:
class_names = load_class_names()

In [13]:
images_train, cls_train, labels_train = load_training_data()

In [14]:
images_test, cls_test, labels_test = load_test_data()

In [15]:
# placeholder variable for the input images
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')
# placeholder variable for true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
#placeholder variable for the class-number
y_true_cls = tf.argmax(y_true, axis=1)

In [16]:
img_size_cropped = 24
#This function takes a single image as input, and a boolean whether to build the training or testing graph.
def pre_process_image(image, training):
    if training:
        # For training, add the following to the TensorFlow graph.
        # Randomly crop the input image.
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])

        # Randomly flip the image horizontally.
        image = tf.image.random_flip_left_right(image)
        
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        # Some of these functions may overflow and result in pixel
        # values beyond the [0, 1] range. It is unclear from the
        # documentation of TensorFlow 0.10.0rc0 whether this is
        # intended. A simple solution is to limit the range.

        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    else:
        # For testing, add the following to the TensorFlow graph.

        # Crop the input image around the centre so it is the same
        # size as images that are randomly cropped during training.
        image = tf.image.resize_image_with_crop_or_pad(image,target_height=img_size_cropped,target_width=img_size_cropped)

    return image

In [17]:
 # Use TensorFlow to loop over all the input images and call the function above which takes a single image as input.
def pre_process(images, training):
    images = tf.map_fn(lambda image: pre_process_image(image, training), images)
    return images

In [18]:
#Set the network architecture
def main_network(images, training):
    # Wrap the input images as a Pretty Tensor object.
    x_pretty = pt.wrap(images)
    # Pretty Tensor uses special numbers to distinguish between the training and testing phases.
    if training:
        phase = pt.Phase.train
    else:
        phase = pt.Phase.infer
    # Create the convolutional neural network of the recommended architecture
    with pt.defaults_scope(activation_fn=tf.nn.relu, phase=phase):
        y_pred, loss = x_pretty.\
            conv2d(kernel=5, depth=64, name='layer_conv1', batch_normalize=True).\
            max_pool(kernel=2, stride=2).\
            conv2d(kernel=5, depth=64, name='layer_conv2').\
            max_pool(kernel=2, stride=2).\
            flatten().\
            fully_connected(size=256, name='layer_fc1').\
            fully_connected(size=128, name='layer_fc2').\
            softmax_classifier(num_classes=num_classes, labels=y_true)
    return y_pred, loss

In [19]:
def create_network(training):
    # Wrap the neural network in the scope named 'network'.
    # Create new variables during training, and re-use during testing.
    with tf.variable_scope('network', reuse=not training):
        # Just rename the input placeholder variable for convenience.
        images = x
        # Create TensorFlow graph for pre-processing.
        images = pre_process(images=images, training=training)
        # Create TensorFlow graph for the main processing.
        y_pred, loss = main_network(images=images, training=training)
    return y_pred, loss

In [20]:
#keeps track of iteration number
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)

In [21]:
_, loss = create_network(training=True)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)   

In [23]:
#Get predicted vector from network
y_pred, _ = create_network(training=False)
#Get predicted class from the vector
y_pred_cls = tf.argmax(y_pred, axis=1)
#Get vector of booleans representing if predicted class is equal to actual class
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
#Get classification accurscy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
#Save variales of graph
saver = tf.train.Saver()
session = tf.Session()
save_dir = 'checkpoints/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_path = os.path.join(save_dir, 'cifar10_cnn')

In [28]:
try:
    # Use TensorFlow to find the latest checkpoint - if any.
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)
    # Try and load the data in the checkpoint.
    saver.restore(session, save_path=last_chk_path)
    # If we get to this point, the checkpoint was successfully loaded.
    print("Restored checkpoint from:", last_chk_path)
except:
    # If the above failed for some reason, simply initialize all the variables for the TensorFlow graph.
    print("Failed to restore checkpoint. Initializing variables instead.")
    session.run(tf.global_variables_initializer())

Trying to restore last checkpoint ...
Failed to restore checkpoint. Initializing variables instead.


In [29]:
#Minibatch training
train_batch_size = 64
def random_batch():
    # Number of images in the training-set.
    num_images = len(images_train)

    # Create a random index.
    idx = np.random.choice(num_images,
                           size=train_batch_size,
                           replace=False)

    # Use the random index to select random images and labels.
    x_batch = images_train[idx, :, :, :]
    y_batch = labels_train[idx, :]

    return x_batch, y_batch

In [30]:
#To store test & training batch accuracy after model has been trained for every 100 iterations
global_arr=[]
batch_acc_arr=[]
test_acc_arr=[]

In [32]:
#Training the network
def optimize(num_iterations):
    # Start-time used for printing time-usage below.    
    start_time = time.time()

    for i in range(num_iterations):
        # Get a batch of training examples. x_batch now holds a batch of images and y_true_batch are the true labels for those images.
        x_batch, y_true_batch = random_batch()
        # Put the batch into a dict with the proper names for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch, y_true: y_true_batch}
        # Run the optimizer using this batch of training data. TensorFlow assigns the variables in feed_dict_train to the placeholder variables and then runs the optimizer.
        # We also want to retrieve the global_step counter.
        i_global, _ = session.run([global_step, optimizer], feed_dict=feed_dict_train)

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            # Calculate the accuracy on the training-batch.
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))
            global_arr.append(i_global)
            batch_acc_arr.append(batch_acc)
            test_acc_arr.append(print_test_accuracy())

        # Save a checkpoint to disk every 1000 iterations (and last).
        if (i_global % 1000 == 0) or (i == num_iterations - 1):
            # Save all variables of the TensorFlow graph to a checkpoint. Append the global_step counter to the filename so we save the last several checkpoints.
            saver.save(session, save_path=save_path, global_step=global_step)         
            print("Saved checkpoint.")

    # Ending time.
    end_time = time.time()
    # Difference between start and end-times.
    time_dif = end_time - start_time
    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))
    

In [33]:
# Split the data-set in batches of this size to limit RAM usage.
batch_size = 256
# Calculates the predicted classes of images and also returns a boolean array whether the classification of each image is correct
def predict_cls(images, labels, cls_true):
    # Number of images.
    num_images = len(images)

    # Allocate an array for the predicted classes which will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_images, dtype=np.int)

    # Now calculate the predicted classes for the batches. We will just iterate through all the batches. There might be a more clever and Pythonic way of doing this.
    # The starting index for the next batch is denoted i.
    i = 0
    while i < num_images:
        # The ending index for the next batch is denoted j.
        j = min(i + batch_size, num_images)
        # Create a feed-dict with the images and labels  between index i and j.
        feed_dict = {x: images[i:j, :], y_true: labels[i:j, :]}
        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)
        # Set the start-index for the next batch to the end-index of the current batch.
        i = j
    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)
    return correct, cls_pred

In [34]:
#Calculate the predicted class for the test-set.
def predict_cls_test():
    return predict_cls(images = images_test, labels = labels_test, cls_true = cls_test)

In [35]:
#Printing the classification accuracy on the test-set
def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # For all the images in the test-set,
    # calculate the predicted classes and whether they are correct.
    correct, cls_pred = predict_cls_test()
    
    # Classification accuracy and the number of correct classifications.
    acc, num_correct = classification_accuracy(correct)
    
    # Number of images being classified.
    num_images = len(correct)

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, num_correct, num_images))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)
        
    return acc

In [36]:
#calculates the classification accuracy 
def classification_accuracy(correct):
    # Return the classification accuracy and the number of correct classifications.
    return correct.mean(), correct.sum()

In [37]:
#Start training
optimize(num_iterations=4000)

Global Step:    100, Training Batch Accuracy:  32.8%
Global Step:    200, Training Batch Accuracy:  32.8%
Global Step:    300, Training Batch Accuracy:  37.5%
Global Step:    400, Training Batch Accuracy:  48.4%
Global Step:    500, Training Batch Accuracy:  48.4%
Global Step:    600, Training Batch Accuracy:  46.9%
Global Step:    700, Training Batch Accuracy:  34.4%
Global Step:    800, Training Batch Accuracy:  48.4%
Global Step:    900, Training Batch Accuracy:  43.8%
Global Step:   1000, Training Batch Accuracy:  39.1%
Saved checkpoint.
Global Step:   1100, Training Batch Accuracy:  37.5%
Global Step:   1200, Training Batch Accuracy:  42.2%
Global Step:   1300, Training Batch Accuracy:  53.1%
Global Step:   1400, Training Batch Accuracy:  37.5%
Global Step:   1500, Training Batch Accuracy:  46.9%
Global Step:   1600, Training Batch Accuracy:  43.8%
Global Step:   1700, Training Batch Accuracy:  39.1%
Global Step:   1800, Training Batch Accuracy:  56.2%
Global Step:   1900, Trainin

In [38]:
#Get final accuracy on test dataset
print_test_accuracy()

Accuracy on Test-Set: 55.5% (5550 / 10000)


0.55500000000000005

In [ ]:
#Plot training accuracy vs iterations 
plt.plot(global_arr, batch_acc_arr, 'r-', label="Training Batch Accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Iterations')
plt.legend()
plt.show()
#Plot tests accuracy vs iterations
plt.plot(global_arr, test_acc_arr, 'b-', label="Test Dataset Accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Iterations')
plt.legend()
plt.show()